In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import to_date, col, date_format
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
import matplotlib.pyplot as plt
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config("spark.driver.memory", "14g") \
    .appName('my-cool-app') \
    .getOrCreate()

sc=spark.sparkContext

23/02/10 07:33:43 WARN Utils: Your hostname, abdi-VivoBook-ASUSLaptop-X415EP-R465EP resolves to a loopback address: 127.0.1.1; using 172.27.172.43 instead (on interface wlo1)
23/02/10 07:33:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/10 07:33:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/10 07:33:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
schema = StructType([ \
        StructField("DEVICE_CODE", StringType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [3]:
df=spark.read.csv('SampleData_Wednesday.csv',header=True,schema=schema)
df.show(1)

23/02/10 07:33:48 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 9, schema size: 7
CSV file: file:///home/abdi/BigData/Project/SampleData_Wednesday.csv/part-00000-d369a0f2-087a-4e02-b4b8-c5f2b9563970-c000.csv
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     900174|      283|       91843693|     91843693|               5|       264|2021-12-29 07:15:35|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



In [4]:
#adding day of week column to dataframe
df = df.withColumn('date', to_date(df.PASS_DAY_TIME))
df = df.withColumn("day_of_week", date_format(col("date"), "EEEE"))

In [5]:
distinct_cameras = [x.DEVICE_CODE for x in df.select('DEVICE_CODE').distinct().collect()]

In [6]:
distinct_cameras = sorted(distinct_cameras)
distinct_cameras

['100',
 '1001001',
 '1001002',
 '1001003',
 '1001004',
 '1001007',
 '1001008',
 '1001009',
 '1001010',
 '1001011',
 '1001012',
 '1001016',
 '1001017',
 '1001018',
 '1001022',
 '1001023',
 '1001024',
 '1001031',
 '1001038',
 '1001040',
 '1001042',
 '1001043',
 '1001044',
 '1001047',
 '1001048',
 '1001051',
 '1001060',
 '1001061',
 '1001063',
 '1001073',
 '1001074',
 '1001078',
 '1001079',
 '10012301',
 '10012805',
 '10012806',
 '10015001',
 '10015002',
 '10015101',
 '10015102',
 '10015201',
 '10015301',
 '10015302',
 '100700804',
 '100700812',
 '100700814',
 '100700818',
 '100700820',
 '100700832',
 '100700834',
 '100700835',
 '100700836',
 '100700838',
 '100700839',
 '100700841',
 '100700847',
 '100700853',
 '100700857',
 '100700861',
 '100700862',
 '100700864',
 '100700871',
 '100700880',
 '100700881',
 '100700894',
 '100700898',
 '100700899',
 '10070090',
 '100700901',
 '100700902',
 '100700905',
 '100700910',
 '100700911',
 '100700914',
 '100700918',
 '100700919',
 '100700920',
 '1

In [16]:
len(distinct_cameras)

923

In [7]:
Traffic = df.rdd.map(lambda x: ((x['FINAL_CAR_KEY'] , x['DEVICE_CODE']),1))


In [8]:
from operator import add
Traffic = Traffic.reduceByKey(add)

In [9]:
Traffic = Traffic.sortBy(lambda x : -x[1])

23/02/10 07:33:51 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 9, schema size: 7
CSV file: file:///home/abdi/BigData/Project/SampleData_Wednesday.csv/part-00000-d369a0f2-087a-4e02-b4b8-c5f2b9563970-c000.csv


In [10]:
all_pair_list = Traffic.collect()
all_pair_list

[(('64111706', '22010054'), 23070),
 (('64111706', '22010039'), 19338),
 (('64111706', '22010053'), 18794),
 (('64111706', '22010040'), 14175),
 (('64111706', '22010061'), 12505),
 (('64111706', '22010047'), 9532),
 (('64111706', '100701263'), 9290),
 (('64111706', '22010083'), 9100),
 (('64111706', '22010048'), 8553),
 (('64111706', '100701299'), 7658),
 (('64111706', '22010060'), 7405),
 (('64111706', '100701119'), 7178),
 (('64111706', '100701265'), 6600),
 (('64111706', '900244'), 6142),
 (('64111706', '22010078'), 6141),
 (('64111706', '100701266'), 5943),
 (('64111706', '100701268'), 5794),
 (('64111706', '22010044'), 5534),
 (('64111706', '114'), 5266),
 (('64111706', '631795'), 5199),
 (('64111706', '631357'), 4971),
 (('64111706', '175'), 4797),
 (('64111706', '900225'), 4691),
 (('64111706', '22010059'), 4639),
 (('64111706', '100701302'), 4547),
 (('64111706', '22010079'), 4367),
 (('64111706', '22010084'), 4244),
 (('64111706', '100700841'), 4115),
 (('64111706', '22010080'

In [11]:
Traffic = Traffic.map(lambda x : (x[0][0] , (x[0][1] , x[1])))


In [12]:
Traffic.take(5)

[('64111706', ('22010054', 23070)),
 ('64111706', ('22010039', 19338)),
 ('64111706', ('22010053', 18794)),
 ('64111706', ('22010040', 14175)),
 ('64111706', ('22010061', 12505))]

In [13]:
temp_ter = Traffic
temp_ter = temp_ter.groupByKey()

In [14]:
temp_traffic = temp_ter.map(lambda x : (x[0],sorted(set(list(x[1])))))

In [21]:
def add_all_camera_to_cars(x):
    temp_list = []
    x_list = x[1]
    for item in x[1]:
        temp_list.append(item[0])
    for item in distinct_cameras:
        if item in temp_list:
            pass
        else:
            x_list.append((item , 0))
    return (x[0] , sorted(set(x_list)))
            

In [22]:
temp_traffic = temp_traffic.map(lambda x : add_all_camera_to_cars(x))

In [25]:
all_list = temp_traffic.collect()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 8) / 8]
Traceback (most recent call last):
  File "/home/abdi/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/abdi/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
Matrix_utility = {}
for item in all_list:
    Matrix_utility[item[0]] = item[1]



In [ ]:
import numpy as np
u, s, v = np.linalg.svd(Matrix_utility, full_matrices=True)